In [1]:
# Import the converted model's class
import numpy as np
import random
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from vggnet import VGG_FACE_16_layers as MyNet

# load image and labels
X = np.load("../../outfile_x.npy")
y = np.load("../../outfile_y.npy").astype(int)
n_class= len(np.unique(y))
print "It's a {}-class classification problem".format(str(n_class))
print "====The shape of data : X : {} , y : {}====".format(str(X.shape),str(y.shape))


It's a 5-class classification problem
====The shape of data : X : (7799, 32, 32, 3) , y : (7799,)====


In [2]:
from collections import Counter
Counter(y)
# since 1 is to many sample 1000 out

Counter({0: 275, 1: 6568, 2: 326, 3: 365, 4: 265})

In [3]:
label1=np.where(y==1)
np.random.shuffle(label1)
others=np.where(y!=1)
label_list=list(label1[0][:4000])+list(others[0])

In [4]:
# limit number of label1
X=X[label_list]
y=y[label_list]
print X.shape
print y.shape

(5231, 32, 32, 3)
(5231,)


In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.0001)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial)

# [Sec1] fintune from freeze pool5

In [6]:
images_raw = tf.placeholder(tf.float32, shape=[None, 32,32,3])
images = tf.image.resize_images(images_raw,(224,224))

# concate network
net = MyNet({'data': images})
pool5 = net.layers['pool5']


In [7]:
with tf.Session() as sess:
    # Load the data
    sess.run(tf.global_variables_initializer())
    net.load('vggnet.npy', sess)
    batch_size=30
    idx=0
    result_list=[]
    while idx*batch_size<len(X):
        range_idx=range(idx*batch_size, min( (idx+1)*batch_size,len(X)))
        np_images =X[range_idx]
        feed = {images_raw: np_images}
        pool5_result = sess.run(pool5, feed_dict=feed)
        result_list.append(pool5_result)
        idx+=1

In [8]:
X_pool5=np.concatenate(result_list, axis=0)

In [9]:
X_pool5.shape

(5231, 7, 7, 512)

# split the data set

In [10]:
arr = np.arange(len(X))
np.random.shuffle(arr)
X=X_pool5[arr]
y=y[arr]
data={
  'X_train': X[:int(len(X)*0.8)],
  'y_train': y[:int(len(X)*0.8)],
  'X_val': X[int(len(X)*0.8):],
  'y_val': y[int(len(X)*0.8):],
}
n_val = len(data['X_val'])
n_train = len(data['X_train'])

print "there are "+ str(data['X_train'].shape[0]) + " images in training set"
print "there are "+ str(data['X_val'].shape[0]) + " images in testing set"

there are 4184 images in training set
there are 1047 images in testing set


In [11]:
tf.reset_default_graph()

In [12]:
Hidden_n=4096
keep_prob = tf.placeholder(tf.float32)
X_pool5_ph = tf.placeholder(tf.float32, shape=[None,7,7,512],name='feature_x')
labels_raw = tf.placeholder(tf.int32, shape=[None,],name='truth_y')
labels = tf.one_hot(labels_raw,n_class)

with tf.name_scope('fintune_whimh'):
    with tf.name_scope('fc6'):
        pool5_flat_size = int(np.prod(X_pool5_ph.shape[1:]))
        pool5_flat = tf.reshape(X_pool5_ph, [-1, pool5_flat_size])
        W_fc6 = weight_variable([pool5_flat_size, Hidden_n])
        b_fc6 = bias_variable([Hidden_n])
        H_fc6=tf.nn.relu(tf.matmul(pool5_flat, W_fc6) + b_fc6) 
        H_fc6 = tf.nn.dropout(H_fc6, keep_prob)
    with tf.name_scope('cross_entropy'):
        #calculate_entropy
        W_fc7 = weight_variable([Hidden_n, n_class])
        b_fc7 = bias_variable([n_class])
        fc7 = tf.matmul(H_fc6, W_fc7) + b_fc7 
        
        predctions = tf.argmax(fc7,1, name='predictions')
        ground_truth = tf.argmax(labels,1)
    with tf.name_scope('regular'):    
        regularizers = (tf.nn.l2_loss(W_fc7) + tf.nn.l2_loss(b_fc7)+
                        (tf.nn.l2_loss(W_fc6) + tf.nn.l2_loss(b_fc6)))
        
correct_prediction = tf.equal(predctions, ground_truth)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc7 ,labels= labels), 0)+regularizers*1e-3
train_op = tf.train.AdamOptimizer(1e-6).minimize(loss)


In [ ]:
batch_size=100
with tf.Session() as sess:
    # Load the data
    sess.run(tf.global_variables_initializer())
    for i in range(7000):
        arr = np.arange(n_train)
        np.random.shuffle(arr)
        np_images, np_labels = (data['X_train'][arr[:batch_size]] ,data['y_train'][arr[:batch_size]])
        feed = {X_pool5_ph: np_images, labels_raw: np_labels, keep_prob:0.4}
        np_loss,acc, _ = sess.run([loss,accuracy, train_op], feed_dict=feed)
        if i % 30 == 0:
            arr = np.arange(n_val)
            np.random.shuffle(arr)
            np_images, np_labels = (data['X_val'] ,data['y_val'])
            feed = {X_pool5_ph: np_images, labels_raw: np_labels, keep_prob:1.0}
            vali_loss, vali_acc = sess.run([loss,accuracy], feed_dict=feed)
            print('Iteration: ', i,"training-acc",acc,"vali-loss", vali_loss, "vali-acc", vali_acc)

('Iteration: ', 0, 'training-acc', 0.25999999, 'vali-loss', 1.6050611, 'vali-acc', 0.76981854)
('Iteration: ', 30, 'training-acc', 0.77999997, 'vali-loss', 1.3698556, 'vali-acc', 0.76981854)
('Iteration: ', 60, 'training-acc', 0.75999999, 'vali-loss', 1.0317177, 'vali-acc', 0.76981854)
('Iteration: ', 90, 'training-acc', 0.78999996, 'vali-loss', 0.86502689, 'vali-acc', 0.76981848)
('Iteration: ', 120, 'training-acc', 0.71000004, 'vali-loss', 0.76324797, 'vali-acc', 0.76981848)
('Iteration: ', 150, 'training-acc', 0.76999992, 'vali-loss', 0.68597698, 'vali-acc', 0.77268386)
('Iteration: ', 180, 'training-acc', 0.80999994, 'vali-loss', 0.6258117, 'vali-acc', 0.81757402)
('Iteration: ', 210, 'training-acc', 0.80000001, 'vali-loss', 0.57759732, 'vali-acc', 0.84431702)
('Iteration: ', 240, 'training-acc', 0.89000005, 'vali-loss', 0.54045975, 'vali-acc', 0.85482335)
('Iteration: ', 270, 'training-acc', 0.84000003, 'vali-loss', 0.50377017, 'vali-acc', 0.85864365)
('Iteration: ', 300, 'trainin

# [Sec2] Finetune End2end
skip sec1 after reading data

In [14]:
tf.reset_default_graph()

In [15]:
arr = np.arange(len(X))
np.random.shuffle(arr)
X=X[arr]
y=y[arr]
data={
  'X_train': X[:int(len(X)*0.8)],
  'y_train': y[:int(len(X)*0.8)],
  'X_val': X[int(len(X)*0.8):],
  'y_val': y[int(len(X)*0.8):],
}
n_val = len(data['X_val'])
n_train = len(data['X_train'])
print "there are "+ str(data['X_train'].shape[0]) + " images in training set"
print "there are "+ str(data['X_val'].shape[0]) + " images in testing set"

there are 4184 images in training set
there are 1047 images in testing set


In [16]:
Hidden_n=1024
keep_prob = tf.placeholder(tf.float32)

images_raw = tf.placeholder(tf.float32, shape=[None, 32,32,3])
images = tf.image.resize_images(images_raw,(224,224))
labels_raw = tf.placeholder(tf.int32, shape=[None,],name='truth_y')
labels = tf.one_hot(labels_raw,n_class)

# concate network
net = MyNet({'data': images})
pool5 = net.layers['pool5']
with tf.name_scope('fintune_whimh'):
    with tf.name_scope('fc6'):
        pool5_flat_size = int(np.prod(pool5.shape[1:]))
        pool5_flat = tf.reshape(pool5, [-1, pool5_flat_size])
        W_fc6 = weight_variable([pool5_flat_size, Hidden_n])
        b_fc6 = bias_variable([Hidden_n])
        H_fc6=tf.nn.relu(tf.matmul(pool5_flat, W_fc6) + b_fc6) 
        H_fc6 = tf.nn.dropout(H_fc6, keep_prob)
    with tf.name_scope('cross_entropy'):
        #calculate_entropy
        W_fc7 = weight_variable([Hidden_n, n_class])
        b_fc7 = bias_variable([n_class])
        fc7 = tf.matmul(H_fc6, W_fc7) + b_fc7 
        
        predctions = tf.argmax(fc7,1, name='predictions')
        ground_truth = tf.argmax(labels,1)
    with tf.name_scope('regular'):    
        regularizers = (tf.nn.l2_loss(W_fc7) + tf.nn.l2_loss(b_fc7)+
                        (tf.nn.l2_loss(W_fc6) + tf.nn.l2_loss(b_fc6)))

correct_prediction = tf.equal(predctions, ground_truth)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc7 ,labels= labels), 0)+regularizers*1e-3
train_op = tf.train.AdamOptimizer(1e-6).minimize(loss)


In [17]:
batch_size=30
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Load the data
    net.load('vggnet.npy', sess)
    for i in range(7000):
        arr = np.arange(len(data['X_train']))
        np.random.shuffle(arr)
        np_images, np_labels = (data['X_train'][arr[:batch_size]] ,data['y_train'][arr[:batch_size]])
        feed = {images_raw: np_images, labels_raw: np_labels, keep_prob:0.4}
        np_loss,acc, _ = sess.run([loss,accuracy, train_op], feed_dict=feed)
        if i % 300 == 0:
            idx = 0
            loss_list = []
            acc_list = []
            while idx*batch_size < n_val :
                range_idx=range(idx*batch_size, min( (idx+1)*batch_size,n_val))
                np_images, np_labels = (data['X_val'][range_idx] ,data['y_val'][range_idx])
                feed = {images_raw: np_images, labels_raw: np_labels, keep_prob:1}
                vali_loss, vali_acc = sess.run([loss,accuracy], feed_dict=feed)
                loss_list.append(vali_loss)
                acc_list.append(vali_acc)
                idx+=1
            vali_loss_mean =  np.mean(loss_list)
            vali_acc_mean =  np.mean(acc_list)
            print('Iteration: ', i,"training-acc",acc,"vali-loss", vali_loss_mean, "vali-acc", vali_acc_mean)

('Iteration: ', 0, 'training-acc', 0.033333335, 'vali-loss', 1.6086761, 'vali-acc', 0.75481492)
('Iteration: ', 300, 'training-acc', 0.86666673, 'vali-loss', 0.52611846, 'vali-acc', 0.83978844)
('Iteration: ', 600, 'training-acc', 0.9666667, 'vali-loss', 0.26976711, 'vali-acc', 0.9189418)
('Iteration: ', 900, 'training-acc', 1.0, 'vali-loss', 0.20543028, 'vali-acc', 0.94941807)
('Iteration: ', 1200, 'training-acc', 0.9666667, 'vali-loss', 0.19890976, 'vali-acc', 0.95132273)
('Iteration: ', 1500, 'training-acc', 0.9666667, 'vali-loss', 0.16905002, 'vali-acc', 0.95216936)
('Iteration: ', 1800, 'training-acc', 1.0, 'vali-loss', 0.17373966, 'vali-acc', 0.95703715)
('Iteration: ', 2100, 'training-acc', 1.0, 'vali-loss', 0.1663563, 'vali-acc', 0.95788366)
('Iteration: ', 2400, 'training-acc', 1.0, 'vali-loss', 0.16175134, 'vali-acc', 0.95978838)
('Iteration: ', 2700, 'training-acc', 1.0, 'vali-loss', 0.16979502, 'vali-acc', 0.95978838)
('Iteration: ', 3000, 'training-acc', 1.0, 'vali-loss', 

### only tune after pool5 without forward

In [18]:
tf.reset_default_graph()

In [19]:
Hidden_n=1024
keep_prob = tf.placeholder(tf.float32)

images_raw = tf.placeholder(tf.float32, shape=[None, 32,32,3])
images = tf.image.resize_images(images_raw,(224,224))
labels_raw = tf.placeholder(tf.int32, shape=[None,],name='truth_y')
labels = tf.one_hot(labels_raw,n_class)

# concate network
net = MyNet({'data': images})
pool5 = net.layers['pool5']
with tf.name_scope('fintune_whimh'):
    with tf.name_scope('fc6'):
        pool5_flat_size = int(np.prod(pool5.shape[1:]))
        pool5_flat = tf.reshape(pool5, [-1, pool5_flat_size])
        W_fc6 = weight_variable([pool5_flat_size, Hidden_n])
        b_fc6 = bias_variable([Hidden_n])
        H_fc6=tf.nn.relu(tf.matmul(pool5_flat, W_fc6) + b_fc6) 
        H_fc6 = tf.nn.dropout(H_fc6, keep_prob)
    with tf.name_scope('cross_entropy'):
        #calculate_entropy
        W_fc7 = weight_variable([Hidden_n, n_class])
        b_fc7 = bias_variable([n_class])
        fc7 = tf.matmul(H_fc6, W_fc7) + b_fc7 
        
        predctions = tf.argmax(fc7,1, name='predictions')
        ground_truth = tf.argmax(labels,1)
    with tf.name_scope('regular'):    
        regularizers = (tf.nn.l2_loss(W_fc7) + tf.nn.l2_loss(b_fc7)+
                        (tf.nn.l2_loss(W_fc6) + tf.nn.l2_loss(b_fc6)))
correct_prediction = tf.equal(predctions, ground_truth)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc7 ,labels= labels), 0)+regularizers*1e-3
#train_op = tf.train.AdamOptimizer(1e-6).minimize(loss)
var_list2=[W_fc6,W_fc7,b_fc6,b_fc7]
train_op = tf.train.AdamOptimizer(1e-6).minimize(loss,var_list=var_list2)

In [21]:
batch_size=30
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
     # Load the layers
    net.load('vggnet.npy', sess)
    for i in range(12000):
        arr = np.arange(len(data['X_train']))
        np.random.shuffle(arr)
        np_images, np_labels = (data['X_train'][arr[:batch_size]] ,data['y_train'][arr[:batch_size]])
        feed = {images_raw: np_images, labels_raw: np_labels, keep_prob:0.4}
        np_loss,acc, _ = sess.run([loss,accuracy, train_op], feed_dict=feed)
        if i % 300 == 0:
            idx = 0
            loss_list = []
            acc_list = []
            while idx*batch_size < n_val :
                range_idx=range(idx*batch_size, min( (idx+1)*batch_size,n_val))
                np_images, np_labels = (data['X_val'][range_idx] ,data['y_val'][range_idx])
                feed = {images_raw: np_images, labels_raw: np_labels, keep_prob:1}
                vali_loss, vali_acc = sess.run([loss,accuracy], feed_dict=feed)
                loss_list.append(vali_loss)
                acc_list.append(vali_acc)
                idx+=1
            vali_loss_mean =  np.mean(loss_list)
            vali_acc_mean =  np.mean(acc_list)
            print('Iteration: ', i,"training-acc",acc,"vali-loss", vali_loss_mean, "vali-acc", vali_acc_mean)

('Iteration: ', 0, 'training-acc', 0.40000004, 'vali-loss', 1.6083723, 'vali-acc', 0.75481492)
('Iteration: ', 300, 'training-acc', 0.70000005, 'vali-loss', 0.78898078, 'vali-acc', 0.75481492)
('Iteration: ', 600, 'training-acc', 0.9333334, 'vali-loss', 0.59451509, 'vali-acc', 0.79978848)
('Iteration: ', 900, 'training-acc', 0.83333337, 'vali-loss', 0.49549621, 'vali-acc', 0.85788375)
('Iteration: ', 1200, 'training-acc', 0.76666677, 'vali-loss', 0.42081538, 'vali-acc', 0.87216938)
('Iteration: ', 1500, 'training-acc', 0.9666667, 'vali-loss', 0.36211753, 'vali-acc', 0.89894187)
('Iteration: ', 1800, 'training-acc', 0.9000001, 'vali-loss', 0.31640065, 'vali-acc', 0.91703707)
('Iteration: ', 2100, 'training-acc', 0.9333334, 'vali-loss', 0.28412145, 'vali-acc', 0.92465615)
('Iteration: ', 2400, 'training-acc', 0.9666667, 'vali-loss', 0.25974858, 'vali-acc', 0.93608463)
('Iteration: ', 2700, 'training-acc', 0.9666667, 'vali-loss', 0.23863435, 'vali-acc', 0.94084656)
('Iteration: ', 3000, '

## We can notice that fine tuning with hole network can reduce the loss more faster, but overfit faster

## but with  pooling5 freezed can  reach a lower loss than whole network
